In [55]:
import pandas as pd
import numpy as np

In [163]:
spd = pd.read_csv('/content/SPD_Crime_Data__2008-Present.csv')
off_start_dt = pd.to_datetime(spd['Offense Start DateTime'], format='%m/%d/%Y %I:%M:%S %p')
spd['start_hour'] = off_start_dt.dt.hour
spd['start_month'] = off_start_dt.dt.month
spd['start_year'] = off_start_dt.dt.year

off_record_dt = pd.to_datetime(spd['Report DateTime'], format='%m/%d/%Y %I:%M:%S %p')
spd['report_hour'] = off_record_dt.dt.hour
spd['report_month'] = off_record_dt.dt.month
spd['report_year'] = off_record_dt.dt.year

spd = spd[spd['start_year'] > 2007]

In [164]:
def is_night(hour):
    return hour > 18 or hour < 8
spd['is_night'] = spd['start_hour'].apply(is_night)

In [165]:
night_data = spd[spd['is_night']]

In [161]:
nh_spd = spd.groupby('MCPP') # neighbourhood groupped spd

In [166]:
spd.groupby('Crime Against Category').get_group('PROPERTY')['Offense Parent Group'].unique()

array(['LARCENY-THEFT', 'ROBBERY',
       'DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY', 'FRAUD OFFENSES',
       'COUNTERFEITING/FORGERY', 'EXTORTION/BLACKMAIL',
       'MOTOR VEHICLE THEFT', 'STOLEN PROPERTY OFFENSES', 'EMBEZZLEMENT',
       'ARSON', 'BAD CHECKS', 'BRIBERY', 'BURGLARY/BREAKING&ENTERING'],
      dtype=object)

In [ ]:
#night_data[night_data['Offense Parent Group']=='LARCENY-THEFT'][['start_hour','is_night']]

In [75]:
x = spd.groupby('100 Block Address').count()
d = x[x['Offense Parent Group'] > 100].to_dict()

In [ ]:
d['Longitude']

In [45]:
with open('add.txt', 'w') as f:
  for x in list(spd['100 Block Address'].unique()):
    f.write(str(x)+'\n') 

In [167]:
crime_by_night = night_data.groupby('Offense Parent Group').count() / spd.groupby('Offense Parent Group').count()

crime_by_night['night_percent'] = crime_by_night['Report Number']
crime_by_night['night_percent'].to_csv('crime_by_night.csv') 

In [ ]:
night_pp = night_data.groupby('MCPP').count() / spd.groupby('MCPP').count()
night_pp['night_modifier'] = night_pp['is_night']
#night_pp = night_index.dropna()['is_night'].count()

mean_npp =  night_pp.dropna()['is_night'].mean()
#chagne cxv header
night_index = night_pp.dropna() / mean_npp - 1
night_index = night_index['night_modifier'] 
night_index = night_index.sort_values(ascending=False)
night_index.to_csv('night_modifier.csv')
night_pp['is_night'].mean()
night_index

In [179]:
c = spd.dropna()
c['ci'] = c['Report Number']
total_ci = c['ci'].count()
area_pp = c.groupby('MCPP').count()['ci'] / total_ci
area_pp = area_pp / area_pp.mean()
# night_pp['night_index'] = night_pp['is_night']
# #night_pp = night_index.dropna()['is_night'].count()

# mean_npp =  night_pp.dropna()['is_night'].mean()
# #chagne cxv header
# night_index = night_pp.dropna() / mean_npp - 1
# night_index = night_index['night_index'] 
# night_index.to_csv('night_index')
# night_pp['is_night'].mean()
nci = area_pp + night_index
area_pp.sort_values(ascending=False).to_csv('overall_crime_index.csv')
nci.sort_values(ascending=False).to_csv('night_crime_index.csv')

<ipython-input-179-d859ad9c3e8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['ci'] = c['Report Number']


In [176]:
drive = spd.dropna()
of_list =  ['Theft of Motor Vehicle Parts or Accessories',
                              'Driving Under the Influence', 
                              'Theft From Motor Vehicle',
                                                      'Motor Vehicle Theft']

pattern = '|'.join(of_list)       
filter = drive['Offense'].str.match(pattern)                                           
drive = drive[filter]                                          

In [178]:
drive['ci'] = drive['Report Number']
total_ci = drive['ci'].count()
d = drive.groupby('MCPP').count()['ci'] / total_ci
d = d / d.mean()
# night_pp['night_index'] = night_pp['is_night']
# #night_pp = night_index.dropna()['is_night'].count()

# mean_npp =  night_pp.dropna()['is_night'].mean()
# #chagne cxv header
# night_index = night_pp.dropna() / mean_npp - 1
# night_index = night_index['night_index'] 
# night_index.to_csv('night_index')
# night_pp['is_night'].mean()
#nci = area_pp + night_index
d.sort_values(ascending=False).to_csv('car_crime_index.csv')

In [189]:
df = pd.DataFrame()
df['overall_crime_index'] = area_pp.sort_values(ascending=False)
df['night_modifier'] = night_index
df['night_crime_index'] = area_pp  +  night_index
df.to_csv('combined_ci.csv')